In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import statsmodels.api as sm
from scipy import stats

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [2]:
nfl = pd.read_csv('../../data/value_models_combined.csv')
nfl_rolling = pd.read_csv('../../data/value_models_combined_6_game_rolling.csv')

In [3]:
rush_def = nfl[['season', 'team', 'opponent', 'week', 'team_full', 'opponent_full', 'score', 'opponent_score',
          'rush_def_value']]
opposing_off = nfl_rolling[['season', 'week', 'team_full', 
          'rushing_value']]

In [4]:
df = rush_def.merge(opposing_off, left_on=['season', 'week', 'opponent_full'], 
             right_on=['season', 'week', 'team_full'], suffixes = ('','_opponent'))
df = df.drop(columns=['team_full_opponent'])

In [5]:
def value_adjustment(value, adjusting_value, original_value_percentage=.95, adjustment_threshold=.1):
    value_over_expected = value + adjusting_value
    adjusted_value = (value_over_expected*adjustment_threshold) + (value*original_value_percentage)
    adjustment = adjusted_value - value
    
    return adjustment, adjusted_value

df[['rush_def_adjustment', 'rush_def_adjusted_value']] = df.apply(lambda x: value_adjustment(x.rush_def_value, x.rushing_value, 
                                original_value_percentage=.7, adjustment_threshold=.3), axis=1, result_type='expand')

In [6]:
df.to_csv('rush_defense_value_modeling_adjusted_data/rushing_defense_values_with_adjustment.csv')

In [7]:
testing = df.dropna()
testing.sort_values(by='rush_def_adjustment', ascending=False)

,season,team,opponent,week,team_full,opponent_full,score,opponent_score,rush_def_value,rushing_value,rush_def_adjustment,rush_def_adjusted_value
3251,2020,SEA,NE,2,Seattle Seahawks,New England Patriots,35,30,1.198056,2.770271,0.831081,2.029137
3715,2020,BUF,BAL,19,Buffalo Bills,Baltimore Ravens,17,3,-0.256342,2.580224,0.774067,0.517725
768,2015,SEA,DAL,8,Seattle Seahawks,Dallas Cowboys,13,12,-0.549269,2.510055,0.753017,0.203748
4859,2022,SF,PHI,21,San Francisco 49ers,Philadelphia Eagles,7,31,-1.337421,2.439576,0.731873,-0.605549
3712,2020,TEN,BAL,18,Tennessee Titans,Baltimore Ravens,13,20,-2.029076,2.298562,0.689568,-1.339507
...,...,...,...,...,...,...,...,...,...,...,...,...
1520,2016,CIN,HOU,16,Cincinnati Bengals,Houston Texans,10,12,0.525687,-2.209473,-0.662842,-0.137155
3430,2020,ARI,MIA,9,Arizona Cardinals,Miami Dolphins,31,34,0.261688,-2.266542,-0.679963,-0.418275
2171,2018,GB,MIN,2,Green Bay Packers,Minnesota Vikings,29,29,0.290504,-2.449265,-0.734780,-0.444276
928,2015,DET,LA,14,Detroit Lions,Los Angeles Rams,14,21,-0.936258,-2.870248,-0.861074,-1.797333
